# Meterdatalogic – Scenario Engine Test Notebook

This notebook is for exercising and regression-testing the `meterdatalogic.scenario` features:

- Build a small synthetic *canon* dataset.
- Define a simple retail `Plan`.
- Run scenarios:
  - EV only
  - PV only
  - Battery only
  - EV + PV + Battery combined
- Inspect:
  - `df_before` / `df_after`
  - Summaries (`summary_before` / `summary_after`)
  - Monthly cost estimates and deltas (`cost_before`, `cost_after`, `delta`, `explain`)
- Plot a few before/after comparisons.


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

import importlib
import meterdatalogic

from meterdatalogic import types, ingest, pricing, transform, summary, validate, utils
import meterdatalogic.scenario as scenario

meterdatalogic = importlib.reload(meterdatalogic)
scenario = importlib.reload(scenario)

pd.set_option("display.width", 120)
pd.set_option("display.max_columns", 20)


In [ ]:
from zoneinfo import ZoneInfo
from meterdatalogic import canon

tz = canon.DEFAULT_TZ  # "Australia/Brisbane"

# 7 days of 30-min intervals
idx = pd.date_range(
    "2024-01-01",
    "2024-01-07 23:30",
    freq=f"{canon.DEFAULT_CADENCE_MIN}min",
    tz=ZoneInfo(tz),
    name="t_start",
)

nmi = "NMI1234567"

# Simple diurnal profile: low overnight, higher in evening
hour = idx.hour + idx.minute / 60.0
base_kwh = 0.3 + 0.4 * np.exp(-((hour - 19.0) ** 2) / (2 * 3.0**2))  # “peaky” evenings

df_raw = pd.DataFrame(
    {
        "t_start": idx,          # ingest.from_dataframe will pick this up
        "nmi": nmi,
        "channel": "E1",         # → grid_import via CHANNEL_MAP
        "kwh": base_kwh,
    }
).set_index("t_start")

df_raw.head()


In [ ]:
df_canon = ingest.from_dataframe(df_raw, tz=tz)

print(df_canon.index.name, df_canon.index.tz)
df_canon.head()


In [ ]:
# Ensure this passes canon checks
validate.assert_canon(df_canon)

base_summary = summary.summarise(df_canon)
base_summary["meta"], base_summary["energy"]


In [ ]:
from meterdatalogic.types import ToUBand, DemandCharge, Plan

flat_band = ToUBand(
    name="all_times",
    start="00:00",
    end="24:00",
    rate_c_per_kwh=30.0,  # 30c / kWh
)

plan_flat = Plan(
    usage_bands=[flat_band],
    feed_in_c_per_kwh=8.0,  # 8c / kWh export
    demand=None,            # no demand component for this test
    fixed_c_per_day=100.0,  # $1.00/day
)

plan_flat


In [ ]:
from dataclasses import asdict
from meterdatalogic.types import ScenarioResult

def describe_scenario_result(label: str, result: ScenarioResult):
    print(f"\n=== {label} ===")
    print("ΔkWh / cost deltas:")
    for k, v in result.delta.items():
        print(f"  {k}: {v}")

    print("\nExplain:")
    for k, v in result.explain.items():
        print(f"  {k}: {v}")

    if result.cost_before is not None and result.cost_after is not None:
        print("\nCost (monthly) – before:")
        display(result.cost_before)
        print("\nCost (monthly) – after:")
        display(result.cost_after)
        print("\nCost totals:")
        print("  before:", result.cost_before["total"].sum())
        print("  after :", result.cost_after["total"].sum())


In [ ]:
from meterdatalogic.types import EVConfig

ev_cfg = EVConfig(
    daily_kwh=7.0,
    max_kw=7.0,
    window_start="18:00",
    window_end="07:00",
    days="ALL",
    strategy="immediate",
)

res_ev_only = scenario.run(df_canon, ev=ev_cfg, pv=None, battery=None, plan=plan_flat)
describe_scenario_result("EV only", res_ev_only)

res_ev_only.summary_before["energy"], res_ev_only.summary_after["energy"]


In [ ]:
from meterdatalogic.types import PVConfig

pv_cfg = PVConfig(
    system_kwp=6.6,       # 6.6 kW array
    inverter_kw=5.0,      # 5 kW inverter
    loss_fraction=0.15,   # 15% misc losses
    seasonal_scale=None,  # keep simple for now
)

res_pv_only = scenario.run(df_canon, ev=None, pv=pv_cfg, battery=None, plan=plan_flat)
describe_scenario_result("PV only", res_pv_only)

res_pv_only.summary_before["energy"], res_pv_only.summary_after["energy"]


In [ ]:
from meterdatalogic.types import BatteryConfig

bat_cfg = BatteryConfig(
    capacity_kwh=10.0,
    max_kw=5.0,
    round_trip_eff=0.9,
    soc_min=0.10,
    soc_max=0.95,
    strategy="self_consume",
    allow_grid_charge=False,
    allow_export=False,
)

res_bat_only = scenario.run(df_canon, ev=None, pv=None, battery=bat_cfg, plan=plan_flat)
describe_scenario_result("Battery only (no PV/EV)", res_bat_only)


In [ ]:
res_combo = scenario.run(
    df_canon,
    ev=ev_cfg,
    pv=pv_cfg,
    battery=bat_cfg,
    plan=plan_flat,
)

describe_scenario_result("EV + PV + Battery", res_combo)


In [ ]:
def daily_import_export(df: pd.DataFrame) -> pd.DataFrame:
    g = (
        df.reset_index()
        .groupby(["flow", pd.Grouper(key="t_start", freq="1D")])["kwh"]
        .sum()
        .unstack(0)
        .fillna(0.0)
    )
    g.index.name = "day"
    return g

daily_before = daily_import_export(res_combo.df_before)
daily_after = daily_import_export(res_combo.df_after)

display(daily_before.head())
display(daily_after.head())

# Build Plotly figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=daily_before.index,
        y=daily_before.get("grid_import", 0.0),
        mode="lines+markers",
        name="Import – before",
    )
)

fig.add_trace(
    go.Scatter(
        x=daily_after.index,
        y=daily_after.get("grid_import", 0.0),
        mode="lines+markers",
        name="Import – after",
    )
)

fig.update_layout(
    title="Daily grid import – before vs after (EV + PV + Battery)",
    xaxis_title="Day",
    yaxis_title="kWh per day",
    xaxis_tickangle=-45,
)

fig.show()


In [ ]:
# Directly generate EV/PV series using the same helpers the scenario engine uses
idx = df_canon.index
interval_h = utils.interval_hours(df_canon)

ev_series = scenario._apply_ev(idx, ev_cfg, interval_h)
pv_series = scenario._apply_pv(idx, pv_cfg, interval_h)

print("EV total kWh over period:", ev_series.sum())
print("PV total kWh over period:", pv_series.sum())

# EV profile
fig_ev = go.Figure()
fig_ev.add_trace(
    go.Scatter(
        x=ev_series.index,
        y=ev_series.values,
        mode="lines",
        name="EV kWh",
    )
)
fig_ev.update_layout(
    title="EV charging profile",
    xaxis_title="Time",
    yaxis_title="kWh per interval",
    xaxis_tickangle=-45,
)
fig_ev.show()

# PV profile
fig_pv = go.Figure()
fig_pv.add_trace(
    go.Scatter(
        x=pv_series.index,
        y=pv_series.values,
        mode="lines",
        name="PV kWh",
    )
)
fig_pv.update_layout(
    title="PV generation profile",
    xaxis_title="Time",
    yaxis_title="kWh per interval",
    xaxis_tickangle=-45,
)
fig_pv.show()


In [ ]:
test_grid = [
    dict(strategy="immediate", window_start="18:00", window_end="07:00", days="ALL"),
    dict(strategy="immediate", window_start="22:00", window_end="06:00", days="MF"),
    dict(strategy="scheduled", window_start="10:00", window_end="15:00", days="MS"),
]

results = []

for cfg in test_grid:
    ev_cfg_var = EVConfig(
        daily_kwh=7.0,
        max_kw=7.0,
        window_start=cfg["window_start"],
        window_end=cfg["window_end"],
        days=cfg["days"],
        strategy=cfg["strategy"],
    )
    res = scenario.run(df_canon, ev=ev_cfg_var, pv=pv_cfg, battery=bat_cfg, plan=plan_flat)
    rec = {
        "strategy": cfg["strategy"],
        "window": f'{cfg["window_start"]}–{cfg["window_end"]}',
        "days": cfg["days"],
        "import_kwh_delta": res.delta["import_kwh_delta"],
        "export_kwh_delta": res.delta["export_kwh_delta"],
        "cost_total_delta": res.delta["cost_total_delta"],
        "ev_kwh": res.explain["ev_kwh"],
        "pv_self_consumption_pct": res.explain["pv_self_consumption_pct"],
    }
    results.append(rec)

pd.DataFrame(results)
